![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_Hubert.ipynb)

# Import OpenVINO Hubert  models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BGE models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for Hubert  from Hubert  and they have to be in `Automatic Speech Recognition` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) serves as the bridge between Hugging Face’s Transformers library and Intel’s model optimization and acceleration toolkit, OpenVINO. It enables seamless conversion and deployment of Transformer models for efficient inference.

- We use the `optimum-cli` command-line tool to export Hugging Face models directly to the OpenVINO Intermediate Representation (IR) format.
- In this case, we're exporting the [facebook/hubert-large-ls960-ft](https://huggingface.co/facebook/hubert-large-ls960-ft) model, a fine-tuned HuBERT model designed for speech-related tasks.
- The exported model is saved in the specified output directory (`ov_models/facebook/hubert-large-ls960-ft`) in OpenVINO IR format (`.xml` and `.bin` files).
- Tokenizers and feature extractors (e.g., `Wav2Vec2Processor`) are saved separately as assets (typically in an `assets/` subfolder), which are required for input preprocessing in downstream pipelines like Spark NLP.


In [2]:
MODEL_NAME = "facebook/hubert-large-ls960-ft"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

! optimum-cli export openvino --model {MODEL_NAME} {EXPORT_PATH}

2025-06-22 07:55:49.219687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750578949.499479    1342 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750578949.578059    1342 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-22 07:55:50.152580: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
config.json: 100% 1.38k/1.38k [00:00<00:00, 7.38MB/s]
pytorch_mod

- Saves the Wav2Vec2Processor to the `assets/` folder for audio preprocessing during inference.


In [3]:
from transformers import Wav2Vec2Processor

!mkdir {EXPORT_PATH}/assets

processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
processor.save_pretrained(f"{EXPORT_PATH}/assets")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[]

In [4]:
!ls -l {EXPORT_PATH}/assets

total 16
-rw-r--r-- 1 root root  254 Jun 22 07:57 preprocessor_config.json
-rw-r--r-- 1 root root   96 Jun 22 07:57 special_tokens_map.json
-rw-r--r-- 1 root root 1120 Jun 22 07:57 tokenizer_config.json
-rw-r--r-- 1 root root  358 Jun 22 07:57 vocab.json


## Import and Save Hubert in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 17.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `HubertForCTC` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `HubertForCTC` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [7]:
from sparknlp.annotator import HubertForCTC

hubert = (
    HubertForCTC.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [8]:
hubert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX Hubert model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {MODEL_NAME}_spark_nlp

total 2466448
drwxr-xr-x 3 root root       4096 Jun 22 08:00 fields
-rw-r--r-- 1 root root 1262810979 Jun 22 08:01 hubert_ctc_openvino
drwxr-xr-x 2 root root       4096 Jun 22 08:00 metadata
-rw-r--r-- 1 root root 1262810979 Jun 22 08:01 wav_ctc_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Hubert model 😊

In [11]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt

--2025-06-22 08:01:59--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2199992 (2.1M) [text/plain]
Saving to: ‘librispeech_asr_0.txt’

librispeech_asr_0.t 100%[===================>]   2.10M  --.-KB/s    in 0.03s   

2025-06-22 08:01:59 (79.2 MB/s) - ‘librispeech_asr_0.txt’ saved [2199992/2199992]



In [12]:
import numpy as np
from scipy.io.wavfile import write
from IPython.display import Audio

audio = np.loadtxt("librispeech_asr_0.txt")
write("audio.wav", 16000, (audio * 32767).astype(np.int16))

Audio("audio.wav")

In [13]:
from sparknlp.base import AudioAssembler
from sparknlp.annotator import HubertForCTC
from pyspark.ml import Pipeline

audioAssembler = AudioAssembler() \
    .setInputCol("audio_content") \
    .setOutputCol("audio_assembler")

speechToText = HubertForCTC.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([
    audioAssembler,
    speechToText
])

audio_path = "librispeech_asr_0.txt"
with open(audio_path) as file:
    raw_floats = [float(data) for data in file.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")

model = pipeline.fit(processedAudioFloats)
result = model.transform(processedAudioFloats)

result.select("text.result").show(truncate=False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[MISTER QUILTER IS THE APOSTLE OF THE MIDLE CLASES AND WE ARE GLAD TO WELCOME HIS GOSPEL ]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of Hubert models from HuggingFace 🤗 in Spark NLP 🚀
